In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math

def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    '''값이 1개 존재하면 제거'''
    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

# (1) model1 -> 0 

In [126]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)


# 강한 추세성보이는 칼럼 중 일부 제거
drop_a = ['X_1167','X_1168','X_1169','X_1170','X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)

'''
trainA_31_x,testA_31_x = multi_col(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = multi_col(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = multi_col(train_O_31_x,test_O_31_x)

trainA_31_x,testA_31_x = log(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = log(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = log(train_O_31_x,test_O_31_x)
'''

from sklearn.preprocessing import KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy="quantile", random_state=1414)

features = train_T_31_x.columns[1:]
enc.fit(train_T_31_x[features])
binned_df_train = enc.transform(train_T_31_x[features])
binned_df_test = enc.transform(test_T_31_x[features])
for i, feature in enumerate(features):
    train_T_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    test_T_31_x[f"{feature}_binned"] = binned_df_test[:,i]

enc = KBinsDiscretizer(n_bins=22, encode='ordinal', strategy="quantile", random_state=1414)
features = trainA_31_x.columns[1:]
enc.fit(trainA_31_x[features])
binned_df_train = enc.transform(trainA_31_x[features])
binned_df_test = enc.transform(testA_31_x[features])
for i, feature in enumerate(features):
    trainA_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    testA_31_x[f"{feature}_binned"] = binned_df_test[:,i]

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 3711) (67, 3711) (343, 1079) (239, 1079)


In [ ]:
'''
10% -> 26,7
20% -> 28,5
2 -> 33,4
3 -> 26,5 (4,5)(21,0)
5;10 -> 25,6
2,48 - > 29, 4
'''

'''
17-> 3
9 -> 4
'''

In [4]:
print(train[train['Y_Class']==0]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].min(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==2]['Y_Quality'].min(),'\n')

-0.6442300396904909 
 -0.6441937649548493 
 -0.6257823004654807 
 -0.6255805101477584 



In [16]:
model1 = CatBoostRegressor(random_state=8012,
                           depth=10,
                           verbose=500,
                           iterations=1500,
                           learning_rate=0.055)

In [127]:
model2 = CatBoostRegressor(random_state=2000,
                           depth=6,
                           #eval_metric='MAE',
                           verbose=500,
                           iterations=1000,
                           learning_rate=0.1)

In [89]:
model3 = CatBoostRegressor(random_state=4567,
                           depth=5,
                           verbose=500,
                           iterations=8888,
                           learning_rate=0.008)

In [128]:
model2.fit(trainA_31_x, trainA_31_y)
pred_a = model2.predict(testA_31_x)

0:	learn: 0.0180242	total: 29.4ms	remaining: 29.4s
500:	learn: 0.0000111	total: 10.9s	remaining: 10.9s
999:	learn: 0.0000000	total: 21.8s	remaining: 0us


In [90]:
model3.fit(train_T_31_x, train_T_31_y)
pred_t = model3.predict(test_T_31_x)

0:	learn: 0.0089566	total: 8.72ms	remaining: 1m 17s
500:	learn: 0.0056675	total: 2.4s	remaining: 40.3s
1000:	learn: 0.0037345	total: 4.79s	remaining: 37.7s
1500:	learn: 0.0023693	total: 7.16s	remaining: 35.2s
2000:	learn: 0.0014952	total: 9.52s	remaining: 32.8s
2500:	learn: 0.0009839	total: 11.9s	remaining: 30.4s
3000:	learn: 0.0006607	total: 14.3s	remaining: 28s
3500:	learn: 0.0004427	total: 16.7s	remaining: 25.7s
4000:	learn: 0.0002978	total: 19.2s	remaining: 23.4s
4500:	learn: 0.0002021	total: 21.6s	remaining: 21.1s
5000:	learn: 0.0001363	total: 24.1s	remaining: 18.7s
5500:	learn: 0.0000917	total: 26.6s	remaining: 16.4s
6000:	learn: 0.0000616	total: 29.1s	remaining: 14s
6500:	learn: 0.0000417	total: 31.7s	remaining: 11.6s
7000:	learn: 0.0000282	total: 34.2s	remaining: 9.22s
7500:	learn: 0.0000187	total: 36.8s	remaining: 6.8s
8000:	learn: 0.0000126	total: 39.4s	remaining: 4.36s
8500:	learn: 0.0000087	total: 42s	remaining: 1.91s
8887:	learn: 0.0000064	total: 44s	remaining: 0us


In [55]:
model3.fit(train_O_31_x, train_O_31_y)
pred_o = model3.predict(test_O_31_x)

0:	learn: 0.0060199	total: 4.91ms	remaining: 32.7s
500:	learn: 0.0027121	total: 698ms	remaining: 8.59s
1000:	learn: 0.0012708	total: 1.38s	remaining: 7.79s
1500:	learn: 0.0005999	total: 1.98s	remaining: 6.81s
2000:	learn: 0.0002832	total: 2.57s	remaining: 6s
2500:	learn: 0.0001337	total: 3.17s	remaining: 5.28s
3000:	learn: 0.0000631	total: 3.76s	remaining: 4.59s
3500:	learn: 0.0000298	total: 4.34s	remaining: 3.92s
4000:	learn: 0.0000141	total: 4.93s	remaining: 3.28s
4500:	learn: 0.0000066	total: 5.51s	remaining: 2.65s
5000:	learn: 0.0000031	total: 6.1s	remaining: 2.03s
5500:	learn: 0.0000015	total: 6.7s	remaining: 1.42s
6000:	learn: 0.0000007	total: 7.31s	remaining: 810ms
6500:	learn: 0.0000003	total: 7.91s	remaining: 201ms
6665:	learn: 0.0000003	total: 8.11s	remaining: 0us


In [129]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>-0.6256814053066195), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>-0.6256814053066195), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>-0.6256814053066195), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('kbd.csv',index=False)

In [139]:
aa = pd.read_csv('kbd.csv')

In [140]:
aa['Y_Class'].value_counts()

1    265
0     34
2     11
Name: Y_Class, dtype: int64

In [141]:
aa[aa['Y_Class']==2]

,PRODUCT_ID,Y_Class
9,TEST_009,2
11,TEST_011,2
38,TEST_038,2
78,TEST_078,2
114,TEST_114,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
169,TEST_169,2
183,TEST_183,2


In [108]:
testA_31[testA_31['Y_Class']==2]

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,...,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,Y_quanlity,Y_Class
38,TEST_038,2022-09-25 8:50,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,462.0,447.962264,422.0,NaN,NaN,NaN,NaN,NaN,-0.600943,2


In [54]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

'''
 1.6583913280391964 
 1.6768027925285651 
'''

testA_31.loc[(testA_31['Y_quanlity']<1.6583913280391964  ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>1.6768027925285651   ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<1.6583913280391964  ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>1.6768027925285651   ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<1.6583913280391964  ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>1.6768027925285651  ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('depth.csv',index=False)

In [55]:
dd = pd.read_csv('depth.csv')

In [114]:
dd = pd.read_csv('따로.csv')

In [56]:
dd['Y_Class'].value_counts()/len(dd)

1    0.883871
0    0.100000
2    0.016129
Name: Y_Class, dtype: float64

In [59]:
dd['Y_Class'].value_counts()

1    274
0     31
2      5
Name: Y_Class, dtype: int64

In [60]:
dd[dd['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
160,TEST_160,2
161,TEST_161,2
183,TEST_183,2


In [113]:
dd[dd['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
108,TEST_108,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
169,TEST_169,2
172,TEST_172,2
177,TEST_177,2
179,TEST_179,2


In [55]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<1.709298761129632 ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>1.7110631749716663 ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<1.709298761129632 ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>1.7110631749716663 ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<1.709298761129632 ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>1.7110631749716663 ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('1333.csv',index=False)

In [22]:
testA_31['Y_Class'].values

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1])

In [57]:
test_T_31['Y_Class'].values

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [58]:
pd.read_csv('1333.csv')['Y_Class'].value_counts()

1    267
0     39
2      4
Name: Y_Class, dtype: int64

In [52]:
pd.read_csv('1333.csv')[pd.read_csv('1333.csv')['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
163,TEST_163,2
187,TEST_187,2


# (2) model2 -> 2

In [61]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x+0.22))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

# 강한 추세성보이는 칼럼 중 일부 제거
drop_a = ['X_1167','X_1168','X_1169','X_1170','X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 2107) (67, 2107) (343, 668) (239, 668)


In [62]:
print(train[train['Y_Class']==0]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].min(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==2]['Y_Quality'].min(),'\n')

-0.2942815787001621 
 -0.29425601487159814 
 -0.2812456878203449 
 -0.2811027053669316 



In [36]:
print(train[train['Y_Class']==0]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].min(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==2]['Y_Quality'].min(),'\n')

-0.2942815787001621 
 -0.29425601487159814 
 -0.2812456878203449 
 -0.2811027053669316 



In [38]:
'''model2 = CatBoostRegressor(random_state=1234,
                           depth=4,
                           verbose=500,
                           iterations=2500,
                           learning_rate=0.055)'''

In [40]:
model1.fit(trainA_31_x, trainA_31_y)
pred_a_2 = model1.predict(testA_31_x)

0:	learn: 0.0129084	total: 20.7ms	remaining: 51.8s
500:	learn: 0.0001928	total: 8.41s	remaining: 33.6s
1000:	learn: 0.0000045	total: 16.7s	remaining: 25s
1500:	learn: 0.0000001	total: 24.9s	remaining: 16.5s
2000:	learn: 0.0000000	total: 33.2s	remaining: 8.27s
2499:	learn: 0.0000000	total: 41.8s	remaining: 0us


In [41]:
model1.fit(train_T_31_x, train_T_31_y)
pred_t_2 = model1.predict(test_T_31_x)

0:	learn: 0.0063042	total: 8.39ms	remaining: 21s
500:	learn: 0.0001698	total: 3.41s	remaining: 13.6s
1000:	learn: 0.0000053	total: 6.84s	remaining: 10.2s
1500:	learn: 0.0000002	total: 10.2s	remaining: 6.81s
2000:	learn: 0.0000000	total: 13.7s	remaining: 3.4s
2499:	learn: 0.0000000	total: 17.1s	remaining: 0us


In [42]:
model1.fit(train_O_31_x, train_O_31_y)
pred_o_2 = model1.predict(test_O_31_x)

0:	learn: 0.0041922	total: 2.1ms	remaining: 5.26s
500:	learn: 0.0000039	total: 877ms	remaining: 3.5s
1000:	learn: 0.0000000	total: 1.63s	remaining: 2.44s
1500:	learn: 0.0000000	total: 2.35s	remaining: 1.56s
2000:	learn: 0.0000000	total: 3.06s	remaining: 762ms
2499:	learn: 0.0000000	total: 3.77s	remaining: 0us


In [46]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<-2.589 ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']<-2.589 ), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']<-2.589 ), 'Y_Class'] = 0

In [53]:
testA_31['Y_quality2'] = pred_a_2
test_T_31['Y_quality2'] = pred_t_2
test_O_31['Y_quality2'] = pred_o_2

testA_31.loc[(testA_31['Y_quality2']>-0.2812456878203449 ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quality2']>-0.2812456878203449 ), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quality2']>-0.2812456878203449 ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('따로.csv',index=False)

In [54]:
df = pd.read_csv('따로.csv')

In [60]:
df['Y_Class'].value_counts()#/len(df) # 1234, 1500, 0.33

1    259
0     43
2      8
Name: Y_Class, dtype: int64